In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
import random
from copy import copy

import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from pydantic import BaseModel, Field
from typing import List, Tuple

LANGUAGE = "hebrew"
df = pd.read_csv(f"/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/app/app_data/cleaned/{LANGUAGE}.csv")

In [5]:
df

,q_idx,label,gpt_prediction,gemini_prediction
0,5,פורום ארגוני הפסיכולוגיה הציבורית מזהיר מראיינ...,פורום הארגונים למען הפסיכולוגיה הציבורית מזהיר...,פורום הארגונים למען הפסיכולוגיה הציבורית קורא ...
1,7,בפני השופטת גרסטל באה תביעתו של הקבלן אמיליו ...,"בית-המשפט העליון פסק כי כתבה שפורסמה בעיתון ""ש...",בית המשפט העליון הפך פסיקה של בית המשפט המחוזי...
2,9,"ב""ישראל היום"" אולמרט צריך לעוף | ביתר העיתוני...",הציטוטים העסיסיים ממפלגת העבודה מככבים הבוקר ע...,העיתונים עסקו בהתבטאויות של אהוד ברק ופואד בן ...
3,13,"התוכנית ""תיק תקשורת"" מה-17.7.2008: על הסיקור ה...",בתוכן הטקסט מתואר השפעת התקשורת על העסקה של הש...,"הפרק עוסק בסיקור התקשורתי של עסקת השבויים, תוך..."
4,15,"את הסיפור הזה התניעו אנשי התקשורת, לא השוטרים:...","המאמר מתאר את פרשת התקיפה על סמנכ""לית רשת שירה...",המשטרה התייחסה לתיק התקיפה של אנשי התקשורת כתי...
...,...,...,...,...
238,474,הפרשנים מסכימים כי אובמה לא ינקום בנתניהו | ב...,כותרות העיתונים העבריים מתמקדות היום בנושאים ש...,עיתוני ישראל מתמקדים בבחירת אובמה ובמשמעויותיה...
239,475,מה שמפליא במיוחד בעמדתו של אלון בן-דוד ביחס לפ...,הסיכום הוא: בן-דוד מביע נדיבות להדלפות מידע בט...,"אלון בן-דוד מבקר את פרסום ""הארץ"" על חקירת ההדל..."
240,494,"הנהלת ""מעריב"" משיבה לעתירת ארגון העיתונאים בדב...","הפלט יכול להיות: ""פיטוריו של חגי מטר מ""מעריב"" ...","חברת ""אחוזת הירש ישראל"", בעלת ""מעריב"", טוענת ש..."
241,497,"סיקור הטבח ב""שרלי הבדו"" והתגובות לו הציבו בפני...","העיתון ""ניו-יורק טיימס"" החליט לא לפרסם את הקרי...",הניו-יורק טיימס החליט שלא להציג את הקריקטורה ש...


In [6]:
sample_sumary = df.iloc[0]['gpt_prediction']

In [7]:
sample_sumary

'פורום הארגונים למען הפסיכולוגיה הציבורית מזהיר מפרסום תכנים טראומטיים בתקשורת, שעלולים לגרום לטראומטיזציה ולפגיעה בחוסן הנפשי של הצופים והמרואיינים. הפורום מציע קווים מנחים לראיונות עם ניצולי אירועי טראומה, ומזהיר משאלות ישירות ותיאורי זוועה. הפורום קובע כי נדרשת הקפדה על הצורך לשמור על שלומו של המרואיין ולא לגרום לנזק ממשי או להפרעה תפקודית. הפורום קורא לאיפוק וללקיחת אחריות מצד כלי התקשורת, למען בריאותם וחוסנם של האנשים.'

In [227]:
def remove_random_words(sentences: list, num_words: int):
    punctuation_marks = {".", "?", "!", '``', "''", ','}
    
    # Flatten the sentences into a list of (word, sentence_index, word_index)
    all_words = [
        (word, sentence_idx, word_idx)
        for sentence_idx, sentence in enumerate(sentences)
        for word_idx, word in enumerate(sentence)
        if word not in punctuation_marks
    ]

    # Randomly sample words to remove
    words_to_remove = random.sample(all_words, min(num_words, len(all_words)))

    # Convert sampled words into a set of (sentence_index, word_index) for quick lookup
    indices_to_remove = {
        (sentence_idx, word_idx) for _, sentence_idx, word_idx in words_to_remove
    }

    # Reconstruct the sentences without the removed words
    modified_sentences = [
        [
            word
            for word_idx, word in enumerate(sentence)
            if (sentence_idx, word_idx) not in indices_to_remove
        ]
        for sentence_idx, sentence in enumerate(sentences)
    ]

    return modified_sentences

In [496]:
def replace_with_lemma(sentences: list, num_words: int, language: str):
    lemmatize = LANGUAGE_TO_LEMMATIZER[language]
    punctuation_marks = {".", "?", "!", '``', "''", ','}
    
    # Flatten the sentences into a list of (word, sentence_index, word_index)
    all_words = [
        (word, sentence_idx, word_idx)
        for sentence_idx, sentence in enumerate(sentences)
        for word_idx, word in enumerate(sentence)
        if word not in punctuation_marks
    ]

    # Randomly sample words to lemmatize
    words_to_lemmatize = random.sample(all_words, min(num_words, len(all_words)))
        # Convert sampled words into a set of (sentence_index, word_index) for quick lookup
    
    print(words_to_lemmatize)
    indices_to_lemmatize = {
        (sentence_idx, word_idx) for _, sentence_idx, word_idx in words_to_lemmatize
    }
    # Reconstruct the sentences with lemmatized words
    modified_sentences = [
            [
                lemmatize(word) if (sentence_idx, word_idx) in indices_to_lemmatize and lemmatize(word) is not None else word
                for word_idx, word in enumerate(sentence)
            ]
            for sentence_idx, sentence in enumerate(words_tokenized)
        ]
    return modified_sentences

In [497]:
def detokenize(tokenized_sentences: str) -> str:
    from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
    detokenizer = Detok()
    text = detokenizer.detokenize(tokenized_sentences)
    return text

In [498]:
def swap_entities_in_text(text: str, ner_results: List[Tuple[str, str]], n: int) -> str:
    """
    Randomly swaps N pairs of entities with the same label in the text.

    Args:
        text (str): The input text.
        ner_results (List[Tuple[str, str]]): List of tuples containing labels and entities.
        n (int): Number of pairs to swap.

    Returns:
        str: The text with swapped entities.
    """
    # Group entities by their label
    label_to_entities = {}
    print(ner_results)
    for label, entity in ner_results:
        if label not in label_to_entities:
            label_to_entities[label] = []
        label_to_entities[label].append(entity)

    # Identify labels with at least two entities to swap
    swappable_labels = [label for label, entities in label_to_entities.items() if len(entities) > 1]

    # Limit the number of swaps to the available swappable labels
    n = min(n, len(swappable_labels))

    # Randomly select N labels for swapping
    selected_labels = random.sample(swappable_labels, n)

    # Perform swaps
    for label in selected_labels:
        entities = label_to_entities[label]
        if len(entities) >= 2:
            entity1, entity2 = random.sample(entities, 2)
            # Swap entity1 and entity2 in the text
            text = text.replace(entity1, "<TEMP>")
            text = text.replace(entity2, entity1)
            text = text.replace("<TEMP>", entity2)

    return text

In [511]:
class RemoveRandomWordsConfig(BaseModel):
    enabled: bool = Field(..., description="Whether random word removal is enabled.")
    number: int = Field(0, description="Number of random words to remove.")
class TokenizeConfig(BaseModel):
    should_shuffle: bool = False
    should_remove_sentence: bool = False
    should_remove_random_words: RemoveRandomWordsConfig = Field(
        default_factory=lambda: RemoveRandomWordsConfig(enabled=False, number=0),
        description="Configuration for removing random words."
    )
    should_replace_ner: bool = False
    should_remove_words: bool = False
    should_replace_words_with_lemma_form: RemoveRandomWordsConfig = Field(
    default_factory=lambda: RemoveRandomWordsConfig(enabled=False, number=0),
    description="Configuration for replacing words with lemmas."
    )
    should_replace_ner_with_other: bool = False

In [524]:
config = TokenizeConfig(should_shuffle=True, should_remove_sentence=True, should_remove_random_words={'enabled':True, 'number': 10}, should_replace_words_with_lemma_form={'enabled':True, 'number': 10}, should_replace_ner_with_other=True)

In [12]:
from app.app_data.lemmas.lemmatizers import LANGUAGE_TO_lemmatizer
# from app.app_data.lemmas.ner import LANGUAGE_TO_NER

ModuleNotFoundError: No module named 'pydantic.deprecated'

In [502]:
LANGUAGE = 'hebrew'

In [525]:

from typing import List, Tuple

def swap_entities_in_text(text: str, ner_results: List[Tuple[str, str]], n: int) -> str:
    """
    Randomly swaps N pairs of entities with the same label in the text.

    Args:
        text (str): The input text.
        ner_results (List[Tuple[str, str]]): List of tuples containing labels and entities.
        n (int): Number of pairs to swap.

    Returns:
        str: The text with swapped entities.
    """
    # Group entities by their label
    label_to_entities = {}
    for label, entity in ner_results:
        if label not in label_to_entities:
            label_to_entities[label] = []
        label_to_entities[label].append(entity)

    # Identify labels with at least two entities to swap
    swappable_labels = [label for label, entities in label_to_entities.items() if len(entities) > 1]

    # Limit the number of swaps to the available swappable labels
    n = min(n, len(swappable_labels))

    # Randomly select N labels for swapping
    selected_labels = random.sample(swappable_labels, n)
    
    # Perform swaps
    print(selected_labels)
    for label in selected_labels:
        entities = label_to_entities[label]
        if len(entities) >= 2:
            entity1, entity2 = random.sample(entities, 2)
            # Swap entity1 and entity2 in the text
            text = text.replace(entity1, "<TEMP>")
            text = text.replace(entity2, entity1)
            text = text.replace("<TEMP>", entity2)

    return text


In [15]:
def shuffle_pairs(tokenized_sentences):
    # Iterate through the list with a step of 2 to access pairs
    i = random.randint(-1, len(tokenized_sentences) -2)
    tokenized_sentences[i], tokenized_sentences[i + 1] = tokenized_sentences[i + 1], tokenized_sentences[i]
    return tokenized_sentences

In [532]:
def noise_text(text: str, config: TokenizeConfig) -> str:
    tokenized_sentences = sent_tokenize(text)
    temp_tokenized_sentences = copy(tokenized_sentences)
    number_of_sentences = len(tokenized_sentences)
    
    print(len(tokenized_sentences))
    if config.should_shuffle:
        tokenized_sentences = shuffle_pairs(tokenized_sentences)
    
    if config.should_remove_sentence: 
        tokenized_sentences.pop(random.randrange(len(tokenized_sentences))) 
        
    if config.should_remove_random_words.enabled:
        words_tokenized = [word_tokenize(sent) for sent in tokenized_sentences]
        un_removed_words = remove_random_words(words_tokenized, config.should_remove_random_words.number)
        tokenized_sentences = [TreebankWordDetokenizer().detokenize(sent) for sent in un_removed_words]
        
    if config.should_replace_words_with_lemma_form.enabled:
        words_tokenized = [word_tokenize(sent) for sent in tokenized_sentences]
        un_removed_words = replace_with_lemma(words_tokenized, config.should_replace_words_with_lemma_form.number, LANGUAGE)
        print(f"{un_removed_words}:")
        tokenized_sentences = [TreebankWordDetokenizer().detokenize(sent) for sent in un_removed_words]
    
    if config.should_replace_ner_with_other:
        detokenized = detokenize(tokenized_sentences)
        ner_results = LANGUAGE_TO_NER[LANGUAGE](detokenized)
        text = swap_entities_in_text(detokenized, ner_results, n=3)
    else:
        text = detokenize(tokenized_sentences)
    return text

In [533]:
disrupted_dataset = []
config = TokenizeConfig(should_shuffle=True, should_remove_sentence=True, should_remove_random_words={'enabled':True, 'number': 5}, should_replace_words_with_lemma_form=True, should_replace_ner_with_other=True)
for idx, sample in test_df.iterrows():
    gemini_prediction = sample['gemini_prediction']
    gpt_prediction = sample['gpt_prediction']
    q_idx = sample['q_idx']
    label = sample['label']
    strategy = random.randint(0, 2)
    if strategy == 1:
        gpt_prediction = noise_text(gpt_prediction, config)
    elif strategy == 2:
        gemini_prediction = noise_text(gemini_prediction, config)
    new_sample = sample.to_dict() if strategy == 0 else {
        'gemini_prediction': gemini_prediction,
        'gpt_prediction': gpt_prediction,
        'label': label,
        'q_idx': q_idx
    }
    disrupted_dataset.append(new_sample)

In [534]:
# disrupted_df = pd.DataFrame(disrupted_dataset)
# disrupted_df.to_csv(f"/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/app/app_data/corrupted/{LANGUAGE}.csv")

In [535]:
# Build image
# Upload

In [547]:
text = sample_sumary
tokenized_sentences = sent_tokenize(text)
words_tokenized = [word_tokenize(sent) for sent in tokenized_sentences]
un_removed_words = replace_with_lemma([words_tokenized[0]], config.should_replace_words_with_lemma_form.number, LANGUAGE)

print(words_tokenized)
# noise_text(sample_sumary, config)

[('מזהיר', 0, 5), ('של', 0, 17), ('הצופים', 0, 18), ('הארגונים', 0, 1), ('הנפשי', 0, 16), ('ולפגיעה', 0, 14), ('מפרסום', 0, 6), ('פורום', 0, 0), ('לגרום', 0, 12), ('למען', 0, 2)]


/Users/itaimondshine/PycharmProjects/NLP/Evaluation-Of-Unrepresented-Languages/multilingual/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[['פורום', 'הארגונים', 'למען', 'הפסיכולוגיה', 'הציבורית', 'מזהיר', 'מפרסום', 'תכנים', 'טראומטיים', 'בתקשורת', ',', 'שעלולים', 'לגרום', 'לטראומטיזציה', 'ולפגיעה', 'בחוסן', 'הנפשי', 'של', 'הצופים', 'והמרואיינים', '.'], ['הפורום', 'מציע', 'קווים', 'מנחים', 'לראיונות', 'עם', 'ניצולי', 'אירועי', 'טראומה', ',', 'ומזהיר', 'משאלות', 'ישירות', 'ותיאורי', 'זוועה', '.'], ['הפורום', 'קובע', 'כי', 'נדרשת', 'הקפדה', 'על', 'הצורך', 'לשמור', 'על', 'שלומו', 'של', 'המרואיין', 'ולא', 'לגרום', 'לנזק', 'ממשי', 'או', 'להפרעה', 'תפקודית', '.'], ['הפורום', 'קורא', 'לאיפוק', 'וללקיחת', 'אחריות', 'מצד', 'כלי', 'התקשורת', ',', 'למען', 'בריאותם', 'וחוסנם', 'של', 'האנשים', '.']]


In [557]:
punctuation_marks = {".", "?", "!", '``', "''", ','}

lemmatize = LANGUAGE_TO_LEMMATIZER['hebrew']
sentences = [words_tokenized[0]]
# Flatten the sentences into a list of (word, sentence_index, word_index)
all_words = [
    (word, sentence_idx, word_idx)
    for sentence_idx, sentence in enumerate(sentences)
    for word_idx, word in enumerate(sentence)
    if word not in punctuation_marks
]

# Randomly sample words to lemmatize
words_to_lemmatize = random.sample(all_words, min(5, len(all_words)))
    # Convert sampled words into a set of (sentence_index, word_index) for quick lookup
# 
print(words_to_lemmatize)
indices_to_lemmatize = {
    (sentence_idx, word_idx) for _, sentence_idx, word_idx in words_to_lemmatize
}
# # Reconstruct the sentences with lemmatized words
modified_sentences = [
        [
            lemmatize(word) if (sentence_idx, word_idx) in indices_to_lemmatize and lemmatize(word) is not None else word
            for word_idx, word in enumerate(sentence)
        ]
        for sentence_idx, sentence in enumerate(words_tokenized)
    ]
# return modified_sentences

[('הנפשי', 0, 16), ('תכנים', 0, 7), ('לגרום', 0, 12), ('הפסיכולוגיה', 0, 3), ('של', 0, 17)]


/Users/itaimondshine/PycharmProjects/NLP/Evaluation-Of-Unrepresented-Languages/multilingual/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
sample_sumary

NameError: name 'sample_sumary' is not defined

In [550]:
un_removed_words

[['פורום',
  'ארגון',
  'למען',
  'הפסיכולוגיה',
  'הציבורית',
  'הזהיר',
  'פרסום',
  'תכנים',
  'טראומטיים',
  'בתקשורת',
  ',',
  'שעלולים',
  'גרם',
  'לטראומטיזציה',
  'פגיעה',
  'בחוסן',
  'הנפשי',
  'של',
  'צופה',
  'והמרואיינים',
  '.'],
 ['הפורום',
  'מציע',
  'קווים',
  'מנחים',
  'לראיונות',
  'עם',
  'ניצולי',
  'אירועי',
  'טראומה',
  ',',
  'ומזהיר',
  'משאלות',
  'ישירות',
  'ותיאורי',
  'זוועה',
  '.'],
 ['הפורום',
  'קובע',
  'כי',
  'נדרשת',
  'הקפדה',
  'על',
  'הצורך',
  'לשמור',
  'על',
  'שלומו',
  'של',
  'המרואיין',
  'ולא',
  'לגרום',
  'לנזק',
  'ממשי',
  'או',
  'להפרעה',
  'תפקודית',
  '.'],
 ['הפורום',
  'קורא',
  'לאיפוק',
  'וללקיחת',
  'אחריות',
  'מצד',
  'כלי',
  'התקשורת',
  ',',
  'למען',
  'בריאותם',
  'וחוסנם',
  'של',
  'האנשים',
  '.']]

In [537]:
sample_sumary

'פורום הארגונים למען הפסיכולוגיה הציבורית מזהיר מפרסום תכנים טראומטיים בתקשורת, שעלולים לגרום לטראומטיזציה ולפגיעה בחוסן הנפשי של הצופים והמרואיינים. הפורום מציע קווים מנחים לראיונות עם ניצולי אירועי טראומה, ומזהיר משאלות ישירות ותיאורי זוועה. הפורום קובע כי נדרשת הקפדה על הצורך לשמור על שלומו של המרואיין ולא לגרום לנזק ממשי או להפרעה תפקודית. הפורום קורא לאיפוק וללקיחת אחריות מצד כלי התקשורת, למען בריאותם וחוסנם של האנשים.'

In [539]:
 lemmatize = LANGUAGE_TO_LEMMATIZER['hebrew']

In [541]:
lemmatize("לטראומטיזציה")

/Users/itaimondshine/PycharmProjects/NLP/Evaluation-Of-Unrepresented-Languages/multilingual/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


'אוטומציה'

In [8]:
arabic_lemmer_chache = qalsadi.lemmatizer.Lemmatizer()

NameError: name 'qalsadi' is not defined

In [450]:
def replace_lemma(sentences: list, num_words: int, language: str):
    lemmatize = LANGUAGE_TO_LEMMATIZER[language]
    punctuation_marks = {".", "?", "!", '``', "''", ','}
    
    # Flatten the sentences into a list of (word, sentence_index, word_index)
    all_words = [
        (word, sentence_idx, word_idx)
        for sentence_idx, sentence in enumerate(sentences)
        for word_idx, word in enumerate(sentence)
        if word not in punctuation_marks
    ]

    # Randomly sample words to lemmatize
    words_to_lemmatize = random.sample(all_words, min(num_words, len(all_words)))
    
    print(f"words_to_lemmatize: {len(words_to_lemmatize)}")
    # Convert sampled words into a set of (sentence_index, word_index) for quick lookup
    indices_to_lemmatize = {
        (sentence_idx, word_idx) for _, sentence_idx, word_idx in words_to_lemmatize
    }
    
    # Reconstruct the sentences with lemmatized words
    modified_sentences = [
        [
            lemmatize(word) if (sentence_idx, word_idx) in indices_to_lemmatize else word
            for word_idx, word in enumerate(sentence)
        ]
        for sentence_idx, sentence in enumerate(sentences)
    ]
    return modified_sentences

In [451]:
sample_sumary

'פורום הארגונים למען הפסיכולוגיה הציבורית מזהיר מפרסום תכנים טראומטיים בתקשורת, שעלולים לגרום לטראומטיזציה ולפגיעה בחוסן הנפשי של הצופים והמרואיינים. הפורום מציע קווים מנחים לראיונות עם ניצולי אירועי טראומה, ומזהיר משאלות ישירות ותיאורי זוועה. הפורום קובע כי נדרשת הקפדה על הצורך לשמור על שלומו של המרואיין ולא לגרום לנזק ממשי או להפרעה תפקודית. הפורום קורא לאיפוק וללקיחת אחריות מצד כלי התקשורת, למען בריאותם וחוסנם של האנשים.'

In [452]:
tokenized_sentences = sent_tokenize(sample_sumary)

In [453]:
words_tokenized = [word_tokenize(sent) for sent in tokenized_sentences]
un_removed_words = replace_lemma(words_tokenized, 3, LANGUAGE)

words_to_lemmatize: 3
indices: {(3, 12), (3, 13), (1, 1)}


/Users/itaimondshine/PycharmProjects/NLP/Evaluation-Of-Unrepresented-Languages/multilingual/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


פורום פורום
הארגונים ארגון


KeyboardInterrupt: 

In [454]:
    lemmatize = LANGUAGE_TO_LEMMATIZER['hebrew']
    punctuation_marks = {".", "?", "!", '``', "''", ','}
    
    # Flatten the sentences into a list of (word, sentence_index, word_index)
    all_words = [
        (word, sentence_idx, word_idx)
        for sentence_idx, sentence in enumerate(words_tokenized)
        for word_idx, word in enumerate(sentence)
        if word not in punctuation_marks
    ]

    # Randomly sample words to lemmatize
    words_to_lemmatize = random.sample(all_words, min(5, len(all_words)))
    

In [455]:
words_to_lemmatize

[('להפרעה', 2, 17),
 ('והמרואיינים', 0, 19),
 ('הצורך', 2, 6),
 ('הפורום', 3, 0),
 ('אחריות', 3, 4)]

In [457]:
print(f"words_to_lemmatize: {len(words_to_lemmatize)}")
# Convert sampled words into a set of (sentence_index, word_index) for quick lookup
indices_to_lemmatize = {
    (sentence_idx, word_idx) for _, sentence_idx, word_idx in words_to_lemmatize
}
    

words_to_lemmatize: 5


In [458]:
indices_to_lemmatize

{(0, 19), (2, 6), (2, 17), (3, 0), (3, 4)}

In [463]:
 modified_sentences = [
        [
            lemmatize(word) if (sentence_idx, word_idx) in indices_to_lemmatize and lemmatize(word) is not None else word
            for word_idx, word in enumerate(sentence)
        ]
        for sentence_idx, sentence in enumerate(words_tokenized)
    ]

/Users/itaimondshine/PycharmProjects/NLP/Evaluation-Of-Unrepresented-Languages/multilingual/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [464]:
modified_sentences

[['פורום',
  'הארגונים',
  'למען',
  'הפסיכולוגיה',
  'הציבורית',
  'מזהיר',
  'מפרסום',
  'תכנים',
  'טראומטיים',
  'בתקשורת',
  ',',
  'שעלולים',
  'לגרום',
  'לטראומטיזציה',
  'ולפגיעה',
  'בחוסן',
  'הנפשי',
  'של',
  'הצופים',
  'מרואיין',
  '.'],
 ['הפורום',
  'מציע',
  'קווים',
  'מנחים',
  'לראיונות',
  'עם',
  'ניצולי',
  'אירועי',
  'טראומה',
  ',',
  'ומזהיר',
  'משאלות',
  'ישירות',
  'ותיאורי',
  'זוועה',
  '.'],
 ['הפורום',
  'קובע',
  'כי',
  'נדרשת',
  'הקפדה',
  'על',
  'צורך',
  'לשמור',
  'על',
  'שלומו',
  'של',
  'המרואיין',
  'ולא',
  'לגרום',
  'לנזק',
  'ממשי',
  'או',
  'הפרעה',
  'תפקודית',
  '.'],
 ['פורום',
  'קורא',
  'לאיפוק',
  'וללקיחת',
  'אחריות',
  'מצד',
  'כלי',
  'התקשורת',
  ',',
  'למען',
  'בריאותם',
  'וחוסנם',
  'של',
  'האנשים',
  '.']]